In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import os

In [2]:
os.listdir("../results")

['drug_cells_kernels_best_parameters.csv',
 'X_features_1122_easy_read.txt',
 'drug_cells_PubChem_scaled_kernels_best_parameters.csv',
 '.DS_Store',
 'merged_drug_profiles_cells_sigmoid4_123.csv',
 'drugs_with_pubchem_id.txt',
 'drug_features_with_pubchem_properties.csv',
 'drugs_with_no_pubchem_id.txt',
 'statistics_of_sigmoid_coefficients.csv',
 'target_target_pathway_df.csv',
 'X_features_Targets.txt',
 'X_PubChem_features.txt',
 '.ipynb_checkpoints',
 'X_columns_1094.txt',
 'drug_cells_description_kernels_best_parameters.csv',
 'filtered_drug_profiles_123.csv',
 'X_PubChem_properties.txt',
 'X_features_Target_Pathway.txt',
 'merged_fitted_sigmoid4_123_with_drugs_properties.csv',
 'drug_cells_PubChem_kernels_best_parameters.csv',
 'X_features_cancer_cell_lines.txt']

In [3]:
df_profiles = pd.read_csv("../results/merged_drug_profiles_cells_sigmoid4_123.csv").drop("Unnamed: 0", axis=1)
df_profiles.shape

(2612, 1100)

In [4]:
"MAX_CONC" in df_profiles.columns

True

In [5]:
df_drugs_properties = pd.read_csv("../results/drug_features_with_pubchem_properties.csv").drop(["Unnamed: 0"], axis=1)
df_drugs_properties.shape

(265, 34)

In [6]:
df_targets = pd.read_csv("../results/target_target_pathway_df.csv").drop(["Unnamed: 0"], axis=1)
df_targets.shape

(265, 253)

In [7]:
df_targets.head(2)

,DRUG_ID,LOK,PDK1 (PDPK1),PDGFRB,AURKB,IRAK1,Amyloid beta40,PPARdelta,VEGFR2,ERBB2,...,JNK and p38 signaling,Metabolism,Mitosis,Other,"Other, kinases",PI3K/MTOR signaling,Protein stability and degradation,RTK signaling,WNT signaling,p53 pathway
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [8]:
with open("../results/X_PubChem_properties.txt", 'r') as f:
    X_PubChem_properties = [line.rstrip('\n') for line in f]
print(len(X_PubChem_properties))
difference = set(df_drugs_properties.columns) - set(X_PubChem_properties)
len(difference), difference

26


(8,
 {'DRUG_ID',
  'Drug_Name',
  'PubChem_ID',
  'Synonyms',
  'canonical_smiles',
  'inchi_key',
  'inchi_string',
  'molecular_formula'})

In [9]:
df_drugs_properties.head(2)

,DRUG_ID,Drug_Name,Synonyms,PubChem_ID,molecular_formula,canonical_smiles,inchi_string,inchi_key,molecular_weight,2bonds,...,B,I,Br,Cl,O,N,F,P,S,Pt
0,1,Erlotinib,"Tarceva, RG-1415, CP-358774, OSI-774, Ro-50823...",176870,C22H23N3O4,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,InChI=1S/C22H23N3O4/c1-4-16-6-5-7-17(12-16)25-...,AAKJLRGGTJKAMG-UHFFFAOYSA-N,393.4,8.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,Rapamycin,"AY-22989, Sirolimus, WY-090217, Torisel, Rapamune",5284616,C51H79NO13,CC1CCC2CC(C(=CC=CC=CC(CC(C(=O)C(C(C(=CC(C(=O)C...,InChI=1S/C51H79NO13/c1-30-16-12-11-13-17-31(2)...,QFJCIRLUMZQUOT-HPLJOQBZSA-N,914.2,9.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


### Merge data

In [10]:
column_not_to_use = ["Synonyms", "deriv_found", "PubChem_ID",
                    "inchi_key", "canonical_smiles", "inchi_string"]
columns_to_use = set(df_drugs_properties.columns) - set(column_not_to_use)


merged_df = pd.merge(left = df_profiles, right = df_drugs_properties[columns_to_use], on = "DRUG_ID") 
merged_df = pd.merge(left = merged_df, right = df_targets, on = "DRUG_ID") 
 
merged_df.shape

(2612, 1380)

In [11]:
_FOLDER = "../results/"
# potential features for ML:
# - X_cancer_cell_lines - 1073
# - X_PubChem_properties - 26
# - X_targets - 229
# - X_target_pathway

with open(_FOLDER+"X_features_cancer_cell_lines.txt", 'r') as f:
    X_cancer_cell_lines = [line.rstrip('\n') for line in f]
print("Number of cancer cell lines features:", len(X_cancer_cell_lines))
# *****************************************

with open(_FOLDER+"X_PubChem_properties.txt", 'r') as f:
    X_PubChem_properties = [line.rstrip('\n') for line in f]
print("Number of PubChem drug properties:", len(X_PubChem_properties))
# *****************************************

with open(_FOLDER+"X_features_Targets.txt", 'r') as f:
    X_targets = [line.rstrip('\n') for line in f]
print("Number of possible targets:", len(X_targets))
# *****************************************

with open(_FOLDER+"X_features_Target_Pathway.txt", 'r') as f:
    X_target_pathway = [line.rstrip('\n') for line in f]
print("Number of possible target pathways:", len(X_target_pathway))
# *****************************************

print("\n Maximum number of features:",len(X_cancer_cell_lines)+ len(X_PubChem_properties)+len(X_targets) + len(X_target_pathway))

Number of cancer cell lines features: 1073
Number of PubChem drug properties: 26
Number of possible targets: 229
Number of possible target pathways: 23

 Maximum number of features: 1351


In [12]:
difference = set(merged_df.columns) - set(X_cancer_cell_lines + X_PubChem_properties + X_targets + X_target_pathway)
len(difference), difference

(29,
 {'COSMIC_ID',
  'DRUG_ID',
  'Drug_Name',
  'MAX_CONC',
  'fd_num_0',
  'fd_num_1',
  'fd_num_2',
  'fd_num_3',
  'fd_num_4',
  'fd_num_5',
  'fd_num_6',
  'fd_num_7',
  'fd_num_8',
  'fd_num_9',
  'molecular_formula',
  'norm_cells_0',
  'norm_cells_1',
  'norm_cells_2',
  'norm_cells_3',
  'norm_cells_4',
  'norm_cells_5',
  'norm_cells_6',
  'norm_cells_7',
  'norm_cells_8',
  'norm_cells_9',
  'param_1',
  'param_2',
  'param_3',
  'param_4'})

### Exclude drugs that don't have PubChem_id

In [13]:
drugs_with_pubchem_id = df_drugs_properties[df_drugs_properties["PubChem_ID"]!=0]["DRUG_ID"]
print("All drugs: %d, With known PubChem_Id: %d" % (df_drugs_properties.shape[0], len(drugs_with_pubchem_id)))

All drugs: 265, With known PubChem_Id: 249


In [14]:
drugs_with_pubchem_id = df_drugs_properties[df_drugs_properties["PubChem_ID"]!=0]["DRUG_ID"].values
drugs_with_no_pubchem_id = df_drugs_properties[df_drugs_properties["PubChem_ID"]==0]["DRUG_ID"].values

with open("../results/drugs_with_pubchem_id.txt", 'w') as f:
    for s in drugs_with_pubchem_id:
        f.write(str(s) + '\n')
with open("../results/drugs_with_no_pubchem_id.txt", 'w') as f:
    for s in drugs_with_no_pubchem_id:
        f.write(str(s) + '\n')

In [15]:
filtered_drugs_with_pubchem_id = set(drugs_with_pubchem_id) & set(merged_df["DRUG_ID"].unique())

In [16]:
merged_df = merged_df.set_index("DRUG_ID").loc[filtered_drugs_with_pubchem_id, :].reset_index()
merged_df.shape

(2585, 1380)

### Save the data

In [17]:
merged_df.to_csv("../results/merged_fitted_sigmoid4_123_with_drugs_properties.csv")